<a href="https://colab.research.google.com/github/MisterGrimmeh/applied-crypto-group-proj-1/blob/main/Project1_Problem4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validating Final Solution

In [10]:
import sys
import re

# Function to XOR two hexadecimal strings of the same length
def xor_hex_strings(hex1, hex2):
    bytes1 = bytes.fromhex(hex1)
    bytes2 = bytes.fromhex(hex2)
    return ''.join(chr(b1 ^ b2) for b1, b2 in zip(bytes1, bytes2))

# Function to XOR a ciphertext with a crib to derive a key segment
def derive_key_from_crib(ciphertext_hex, crib, position):
    ciphertext_bytes = bytes.fromhex(ciphertext_hex)
    crib_bytes = crib.encode('utf-8')
    key_segment = ''.join(chr(c ^ crib_bytes[i]) for i, c in enumerate(ciphertext_bytes[position:position + len(crib_bytes)]))
    return key_segment

# Function to XOR message bytes with the key to decrypt the message
def xor_with_key(message_bytes, key_bytes):
    return ''.join(chr(b ^ key_bytes[i % len(key_bytes)]) for i, b in enumerate(message_bytes))

# Function to clean a hexadecimal string by removing invalid characters
def sanitize_and_pad_hex_string(hex_string):
    # Keep only valid hexadecimal characters (0-9, a-f, A-F)
    sanitized = ''.join(c for c in hex_string if c in '0123456789abcdefABCDEF')
    # Length is even by truncating the last character if the length is odd
    if len(sanitized) % 2 != 0:
        sanitized = sanitized[:-1]
    return sanitized

# Function to re-encrypt the decrypted message using the derived key
def encrypt_with_key(plaintext, key_bytes):
    plaintext_bytes = plaintext.encode('utf-8')
    return ''.join(f'{b ^ key_bytes[i % len(key_bytes)]:02x}' for i, b in enumerate(plaintext_bytes))

# All 10 provided ciphertexts
ciphertexts = [
    "71felace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840",
    "71felace559ale7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504ebal2b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdblaf5389719795elf0179cb77c5c456954d88f3",
    "72fe069c51c81a20775928c7879d4fd2a93c3acf3f69fe5fe2e9493a303d9ea98c4e5b60ae40al46058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9",
    "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2fbd6a291571e40balafd3b4blfe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd",
    "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ealece4e31d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aalabc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06",
    "6ef914ce5989152b321a769ad79c42c7be6f6adf2fab19delfc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff82489",
    "71felace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288f5c3a450a527ecc7c82865b8e",
    "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d764983f29bf5",
    "71felace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de029811977430b01748276d79012955cc6a65aebb9054becda5c9278",
    "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7756b2e08aab83bc5560e257"
]

# Target ciphertext to decrypt
target_message_hex = "71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b"

# Sanitize ciphertexts
sanitized_padded_ciphertexts = [sanitize_and_pad_hex_string(ct) for ct in ciphertexts]
sanitized_padded_target_message_hex = sanitize_and_pad_hex_string(target_message_hex)

# Initialize a partial key with null bytes
partial_key = bytearray(len(sanitized_padded_target_message_hex) // 2)

# Apply cribs to construct the partial key
final_message_corrections = {
    "When using a stream cipher": [0],
    "never use the key more than once": [26]
}

for crib, positions in final_message_corrections.items():
    for position in positions:
        key_segment = derive_key_from_crib(sanitized_padded_target_message_hex, crib, position)
        for i, char in enumerate(key_segment):
            partial_key[position + i] = ord(char)

# Decrypt the target message using the fully refined partial key
final_decrypted_message = xor_with_key(bytes.fromhex(sanitized_padded_target_message_hex), partial_key)

# Re-encrypt to verify the decryption
re_encrypted_message_hex = encrypt_with_key(final_decrypted_message, partial_key)

# Compare with the original sanitized target ciphertext
is_encryption_valid = re_encrypted_message_hex == sanitized_padded_target_message_hex

# Output results
print("Final Decrypted Message:")
print(final_decrypted_message)

print("\nValidation Status:")
print("Encryption Valid:", is_encryption_valid)

print("\nRe-encrypted Message Hex:")
print(re_encrypted_message_hex)

print("\nOriginal Target Ciphertext Hex:")
print(sanitized_padded_target_message_hex)

Final Decrypted Message:
When using a stream ciphernever use the key more than once

Validation Status:
Encryption Valid: True

Re-encrypted Message Hex:
71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b

Original Target Ciphertext Hex:
71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b


# Iterative Process

In [ ]:
import sys
import re

# --- Utility Functions ---

def xor_hex_strings(hex1, hex2):
    """XOR two hexadecimal strings of the same length and return the resulting string."""
    bytes1 = bytes.fromhex(hex1)
    bytes2 = bytes.fromhex(hex2)
    return ''.join(chr(b1 ^ b2) for b1, b2 in zip(bytes1, bytes2))

def derive_key_from_crib(ciphertext_hex, crib, position):
    """
    XOR a segment of the ciphertext (starting at the given position) with the crib.
    Returns the derived key segment as a string.
    """
    ciphertext_bytes = bytes.fromhex(ciphertext_hex)
    crib_bytes = crib.encode('utf-8')
    key_segment = ''.join(chr(c ^ crib_bytes[i]) for i, c in enumerate(ciphertext_bytes[position:position + len(crib_bytes)]))
    return key_segment

def xor_with_key(message_bytes, key_bytes):
    """
    Decrypt (or encrypt) a message by XORing it with a repeating key.
    Returns the resulting string.
    """
    return ''.join(chr(b ^ key_bytes[i % len(key_bytes)]) for i, b in enumerate(message_bytes))

def sanitize_and_pad_hex_string(hex_string):
    """
    Remove any non-hex characters from the input and ensure that the string length is even.
    """
    sanitized = ''.join(c for c in hex_string if c in '0123456789abcdefABCDEF')
    if len(sanitized) % 2 != 0:
        sanitized = sanitized[:-1]
    return sanitized

def encrypt_with_key(plaintext, key_bytes):
    """
    Encrypt the plaintext using the key by XORing. Returns the result in hexadecimal.
    """
    plaintext_bytes = plaintext.encode('utf-8')
    return ''.join(f'{b ^ key_bytes[i % len(key_bytes)]:02x}' for i, b in enumerate(plaintext_bytes))

# --- Provided Ciphertexts and Target Message ---

ciphertexts = [
    "71felace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840",
    "71felace559ale7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504ebal2b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdblaf5389719795elf0179cb77c5c456954d88f3",
    "72fe069c51c81a20775928c7879d4fd2a93c3acf3f69fe5fe2e9493a303d9ea98c4e5b60ae40al46058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9",
    "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2fbd6a291571e40balafd3b4blfe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd",
    "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ealece4e31d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aalabc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06",
    "6ef914ce5989152b321a769ad79c42c7be6f6adf2fab19delfc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff82489",
    "71felace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288f5c3a450a527ecc7c82865b8e",
    "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d764983f29bf5",
    "71felace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de029811977430b01748276d79012955cc6a65aebb9054becda5c9278",
    "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7756b2e08aab83bc5560e257"
]

target_message_hex = "71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b"

# --- Sanitize the Ciphertexts ---

sanitized_padded_ciphertexts = [sanitize_and_pad_hex_string(ct) for ct in ciphertexts]
sanitized_padded_target_message_hex = sanitize_and_pad_hex_string(target_message_hex)

# --- Initialize a Partial Key for the Target Message ---
# We create a key of the same length (in bytes) as the target ciphertext.
partial_key = bytearray(len(sanitized_padded_target_message_hex) // 2)

# --- Interactive Crib Dragging Loop ---
def interactive_crib_drag():
    print("Interactive Crib Dragging for Many-Time Pad Cryptanalysis\n")
    print("The target ciphertext is:")
    print(sanitized_padded_target_message_hex)
    while True:
        print("\nCurrent decryption of target message:")
        decrypted_message = xor_with_key(bytes.fromhex(sanitized_padded_target_message_hex), partial_key)
        print(decrypted_message)
        print("\nCurrent key (in hex; unknown bytes are shown as '00'):")
        print(''.join(f'{b:02x}' for b in partial_key))
        print("\nOptions:")
        print("1. Add a crib (update key segment)")
        print("2. Re-encrypt the decrypted message to validate the key")
        print("3. Exit")
        choice = input("Enter choice (1/2/3): ").strip()

        if choice == '1':
            crib = input("Enter crib (known plaintext segment): ").strip()
            pos_input = input("Enter starting position (byte index) for the crib: ").strip()
            try:
                pos = int(pos_input)
            except ValueError:
                print("Invalid position input; please enter an integer.")
                continue
            # Derive the key segment based on the crib guess
            key_segment = derive_key_from_crib(sanitized_padded_target_message_hex, crib, pos)
            print("\nDerived key segment (as string):", key_segment)
            # Update the partial key with the new segment
            for i, char in enumerate(key_segment):
                if pos + i < len(partial_key):
                    partial_key[pos + i] = ord(char)
                else:
                    print("Warning: Crib exceeds key length; truncating.")
                    break
        elif choice == '2':
            # Decrypt using the current key and then re-encrypt to check consistency.
            final_decrypted_message = xor_with_key(bytes.fromhex(sanitized_padded_target_message_hex), partial_key)
            re_encrypted_message_hex = encrypt_with_key(final_decrypted_message, partial_key)
            is_valid = re_encrypted_message_hex == sanitized_padded_target_message_hex
            print("\nRe-encrypted message hex:")
            print(re_encrypted_message_hex)
            print("\nOriginal target ciphertext hex:")
            print(sanitized_padded_target_message_hex)
            print("\nEncryption Valid:", is_valid)
        elif choice == '3':
            print("Exiting interactive crib dragging.")
            break
        else:
            print("Invalid option. Please choose 1, 2, or 3.")

# --- Main Execution ---
if __name__ == "__main__":
    interactive_crib_drag()
